In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

In [2]:
%%time

transactions = pd.read_parquet('transactions_train.parquet')
customers = pd.read_parquet('customers.parquet')
articles = pd.read_parquet('articles.parquet')

CPU times: user 2.86 s, sys: 1.74 s, total: 4.61 s
Wall time: 1.64 s


In [3]:
customers['age_bucket'] = pd.cut(customers['age'].fillna(22), [15, 24, 35, 50, 100], 
                                    labels=['16-24', '25-35', '36-50', '51-100'])

In [4]:
customers.Active.value_counts()

-1    907576
 1    464404
Name: Active, dtype: int64

In [5]:
customers.FN.value_counts()

-1    895050
 1    476930
Name: FN, dtype: int64

In [6]:
customers.club_member_status.value_counts()

 0    1272491
 1      92960
-1       6062
 2        467
Name: club_member_status, dtype: int64

In [7]:
transactions

t_dat           customer_id  article_id     price  \
25784    2018-09-20      1728846800780188   519773001  0.028458   
25785    2018-09-20      1728846800780188   578472001  0.032525   
5389     2018-09-20      2076973761519164   661795002  0.167797   
5390     2018-09-20      2076973761519164   684080003  0.101678   
47429    2018-09-20      2918879973994241   662980001  0.033881   
...             ...                   ...         ...       ...   
31774722 2020-09-22  18439937050817258297   891591003  0.084729   
31774723 2020-09-22  18439937050817258297   869706005  0.084729   
31779097 2020-09-22  18440902715633436014   918894002  0.016932   
31779098 2020-09-22  18440902715633436014   761269001  0.016932   
31780475 2020-09-22  18443633011701112574   914868002  0.033881   

          sales_channel_id  week  
25784                    2     0  
25785                    2     0  
5389                     2     0  
5390                     2     0  
47429                    1     0  
...                    ...   ...  
31774722                 2   104  
31774723                 2   104  
31779097                 1   104  
31779098                 1   104  
31780475                 1   104  

[31788324 rows x 6 columns]

In [8]:
VALID = False

if VALID:
    transactions = transactions[transactions.t_dat<'2020-09-16']

In [9]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [10]:
%%time
c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 29.4 s, sys: 2.11 s, total: 31.6 s
Wall time: 29.1 s


In [11]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 632 ms, sys: 87 ms, total: 719 ms
Wall time: 719 ms


In [12]:
candidates_last_purchase = transactions.copy()

In [13]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week = weeks

CPU times: user 7.45 s, sys: 67.3 ms, total: 7.52 s
Wall time: 7.52 s


In [14]:
transactions[transactions.customer_id==28847241659200]

t_dat     customer_id  article_id     price  sales_channel_id  \
29202087 2020-07-18  28847241659200   762846001  0.025407                 1   
29202088 2020-07-18  28847241659200   829308001  0.033881                 1   
29527049 2020-07-26  28847241659200   887770001  0.016932                 1   
30989557 2020-08-31  28847241659200   760084003  0.025407                 1   
31100629 2020-09-03  28847241659200   925246001  0.128797                 2   

          week  
29202087    95  
29202088    95  
29527049    96  
30989557   101  
31100629   102

In [15]:
candidates_last_purchase[candidates_last_purchase.customer_id==28847241659200]

t_dat     customer_id  article_id     price  sales_channel_id  \
29202087 2020-07-18  28847241659200   762846001  0.025407                 1   
29202088 2020-07-18  28847241659200   829308001  0.033881                 1   
29527049 2020-07-26  28847241659200   887770001  0.016932                 1   
30989557 2020-08-31  28847241659200   760084003  0.025407                 1   
31100629 2020-09-03  28847241659200   925246001  0.128797                 2   

          week  
29202087    96  
29202088    96  
29527049   101  
30989557   102  
31100629   105

In [16]:
transactions = transactions.merge(articles[['article_id','prod_name', 'colour_group_name',
                                            'graphical_appearance_name']],
                                 on='article_id', how='left')

transactions = transactions.merge(customers[['age_bucket', 'customer_id']], on='customer_id', how='left')

In [17]:
transactions.head()

t_dat      customer_id  article_id     price  sales_channel_id  week  \
0 2020-07-15  272412481300040   778064028  0.008458                 1    95   
1 2020-07-15  272412481300040   816592008  0.016932                 1    95   
2 2020-07-15  272412481300040   621381021  0.033881                 1    95   
3 2020-07-15  272412481300040   817477003  0.025407                 1    95   
4 2020-07-15  272412481300040   899088002  0.025407                 1    95   

   prod_name  colour_group_name  graphical_appearance_name age_bucket  
0        283                 11                          3      36-50  
1      20092                  2                          5      36-50  
2        935                  8                          0      36-50  
3       5423                 10                          4      36-50  
4      24059                  0                          0      36-50

In [18]:
## From same channel find alternate item from same prod group with highest number of sale
## From same channel find alternate item of lower price with a high number of sale
## From same channel find alternate item of different color with highest number of sale
## From same channel find alternate item of different grpahical appearance with highest number of sale

In [19]:
## top 12 articles from last 10 weeks based on weekly sales

sales = transactions \
    .groupby(['week','sales_channel_id'])['article_id'].value_counts() \
    .groupby(['week','sales_channel_id']).rank(method='dense', ascending=False) \
    .groupby(['week','sales_channel_id']).head(12).rename('bestseller_rank').astype('int8')

In [20]:
sales

week  sales_channel_id  article_id
95    1                 372860002      1
                        372860001      2
                        816166003      3
                        759871002      4
                        464297007      5
                                      ..
104   2                 929275001      8
                        714790020      9
                        896169005     10
                        934835001     11
                        894780001     12
Name: bestseller_rank, Length: 240, dtype: int8

In [21]:
best_sell_df = sales.reset_index()
best_sell_df = best_sell_df.merge(articles[['article_id','prod_name']], on='article_id', how='left')
best_sell_df['RankType'] = 'MostBought'

In [22]:
## From same channel find alternate item from same prod group with highest number of sale

def find_alternate_items(base_itm, base_channel, base_wk, base_prod):

    item_1 = base_itm
    channel_id = base_channel
    wk_num = base_wk
    prod_nm = base_prod
    
    item_2 = ''
    item_3 = ''
    item_4 = ''
    item_5 = ''

    color_1 = articles[articles.article_id==item_1]['colour_group_name'].iloc[0]
    g_1 = articles[articles.article_id==item_1]['graphical_appearance_name'].iloc[0]

    p1 = transactions[(transactions.article_id==item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num)]['price'].mean()

    filter_df = transactions[(transactions.article_id!=item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num)]

    if len(filter_df)>0:

        item_2 = filter_df.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
        color_2 = articles[articles.article_id==item_2]['colour_group_name'].iloc[0]
        g_2 = articles[articles.article_id==item_2]['graphical_appearance_name'].iloc[0]

        p2 = transactions[(transactions.article_id==item_2) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num)]['price'].mean()

        p_low = min(p1, p2)
        filter_df_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num) & 
                 (transactions.price < p_low)]

        if len(filter_df_2)>0:

            item_3 = filter_df_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
            color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
            g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            filter_df_3 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                     (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.colour_group_name.isin([color_1, color_2, color_3]))]

            if len(filter_df_3)>0:
                item_4 = filter_df_3.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_4 = articles[articles.article_id==item_4]['colour_group_name'].iloc[0]
                g_4 = articles[articles.article_id==item_4]['graphical_appearance_name'].iloc[0]

                filter_df_4 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3, item_4])) & 
                     (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3, g_4]))]

                if len(filter_df_4)>0:
                    item_5 = filter_df_4.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]            

            else:
                filter_df_3_1 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3]))]

                if len(filter_df_3_1)>0:
                    item_4 = filter_df_3_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

        else:
            filter_df_2_1 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.colour_group_name.isin([color_1, color_2]))]

            if len(filter_df_2_1)>0:
                item_3 = filter_df_2_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
                g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            else:
                filter_df_2_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                     (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2]))]

                if len(filter_df_2_2)>0:
                    item_3 = filter_df_2_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

    return item_2, item_3, item_4, item_5


In [23]:
alternate_best_sell_df = pd.DataFrame(columns = best_sell_df.columns)

wk_list = []
sc_list = []
art_list = []
bs_list = []
rt_list = []

for i in range(0, len(best_sell_df)):
    
    bs_itm = best_sell_df['article_id'][i]
    bs_channel = best_sell_df['sales_channel_id'][i]
    bs_wk = best_sell_df['week'][i]
    bs_prod = best_sell_df['prod_name'][i]
    bs_rank = best_sell_df['bestseller_rank'][i]
    
    a,b,c,d = find_alternate_items(bs_itm, bs_channel, bs_wk, bs_prod)

    wk_list.extend([bs_wk]*4)
    sc_list.extend([bs_channel]*4)
    art_list.extend([a,b,c,d])
    bs_list.extend([bs_rank]*4)
    rt_list.extend(['SecondMost', 'LowerPrice', 'AnotherColor', 'AnotherAppearance'])

In [24]:
alternate_best_sell_df['week'] = wk_list
alternate_best_sell_df['sales_channel_id'] = sc_list
alternate_best_sell_df['article_id'] = art_list
alternate_best_sell_df['bestseller_rank'] = bs_list
alternate_best_sell_df['RankType'] = rt_list

In [25]:
best_sell_df = pd.concat([best_sell_df.drop('prod_name', axis=1), 
                          alternate_best_sell_df.drop('prod_name', axis=1)], axis=0)

best_sell_df = best_sell_df[~(best_sell_df.article_id=='')] \
                .sort_values(['week','sales_channel_id','bestseller_rank'])

In [26]:
best_sell_df.to_parquet('best_sell_with_channel.parquet')

In [27]:
## From same age_bucket find alternate item from same prod group with highest number of sale
## From same age_bucket find alternate item of lower price with a high number of sale
## From same age_bucket find alternate item of different color with highest number of sale
## From same age_bucket find alternate item of different grpahical appearance with highest number of sale

In [28]:
transactions.head()

t_dat      customer_id  article_id     price  sales_channel_id  week  \
0 2020-07-15  272412481300040   778064028  0.008458                 1    95   
1 2020-07-15  272412481300040   816592008  0.016932                 1    95   
2 2020-07-15  272412481300040   621381021  0.033881                 1    95   
3 2020-07-15  272412481300040   817477003  0.025407                 1    95   
4 2020-07-15  272412481300040   899088002  0.025407                 1    95   

   prod_name  colour_group_name  graphical_appearance_name age_bucket  
0        283                 11                          3      36-50  
1      20092                  2                          5      36-50  
2        935                  8                          0      36-50  
3       5423                 10                          4      36-50  
4      24059                  0                          0      36-50

In [29]:
## top 12 articles from last 10 weeks based on weekly sales

sales = transactions \
    .groupby(['week','age_bucket'])['article_id'].value_counts() \
    .groupby(['week','age_bucket']).rank(method='dense', ascending=False) \
    .groupby(['week','age_bucket']).head(12).rename('bestseller_rank').astype('int8')

In [30]:
best_sell_df_ab = sales.reset_index()
best_sell_df_ab = best_sell_df_ab.merge(articles[['article_id','prod_name']], on='article_id', how='left')
best_sell_df_ab['RankType'] = 'MostBought'

In [31]:
## From same age bucket find alternate items based on second most bought, lower price, another color/appearance

def find_alternate_items_ab(base_itm, base_age_bucket, base_wk, base_prod):

    item_1 = base_itm
    age_bkt = base_age_bucket
    wk_num = base_wk
    prod_nm = base_prod
    
    item_2 = ''
    item_3 = ''
    item_4 = ''
    item_5 = ''

    color_1 = articles[articles.article_id==item_1]['colour_group_name'].iloc[0]
    g_1 = articles[articles.article_id==item_1]['graphical_appearance_name'].iloc[0]

    p1 = transactions[(transactions.article_id==item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num)]['price'].mean()

    filter_df = transactions[(transactions.article_id!=item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num)]

    if len(filter_df)>0:

        item_2 = filter_df.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
        color_2 = articles[articles.article_id==item_2]['colour_group_name'].iloc[0]
        g_2 = articles[articles.article_id==item_2]['graphical_appearance_name'].iloc[0]

        p2 = transactions[(transactions.article_id==item_2) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num)]['price'].mean()

        p_low = min(p1, p2)
        filter_df_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num) & 
                 (transactions.price < p_low)]

        if len(filter_df_2)>0:

            item_3 = filter_df_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
            color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
            g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            filter_df_3 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                     (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.colour_group_name.isin([color_1, color_2, color_3]))]

            if len(filter_df_3)>0:
                item_4 = filter_df_3.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_4 = articles[articles.article_id==item_4]['colour_group_name'].iloc[0]
                g_4 = articles[articles.article_id==item_4]['graphical_appearance_name'].iloc[0]

                filter_df_4 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3, item_4])) & 
                     (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3, g_4]))]

                if len(filter_df_4)>0:
                    item_5 = filter_df_4.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]            

            else:
                filter_df_3_1 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3]))]

                if len(filter_df_3_1)>0:
                    item_4 = filter_df_3_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

        else:
            filter_df_2_1 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.colour_group_name.isin([color_1, color_2]))]

            if len(filter_df_2_1)>0:
                item_3 = filter_df_2_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
                g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            else:
                filter_df_2_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                     (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2]))]

                if len(filter_df_2_2)>0:
                    item_3 = filter_df_2_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

    return item_2, item_3, item_4, item_5


In [32]:
alternate_best_sell_df_ab = pd.DataFrame(columns = best_sell_df_ab.columns)

wk_list = []
ab_list = []
art_list = []
bs_list = []
rt_list = []

for i in range(0, len(best_sell_df_ab)):
    
    bs_itm = best_sell_df_ab['article_id'][i]
    bs_ab = best_sell_df_ab['age_bucket'][i]
    bs_wk = best_sell_df_ab['week'][i]
    bs_prod = best_sell_df_ab['prod_name'][i]
    bs_rank = best_sell_df_ab['bestseller_rank'][i]
    
    a,b,c,d = find_alternate_items_ab(bs_itm, bs_ab, bs_wk, bs_prod)

    wk_list.extend([bs_wk]*4)
    ab_list.extend([bs_ab]*4)
    art_list.extend([a,b,c,d])
    bs_list.extend([bs_rank]*4)
    rt_list.extend(['SecondMost', 'LowerPrice', 'AnotherColor', 'AnotherAppearance'])

In [33]:
alternate_best_sell_df_ab['week'] = wk_list
alternate_best_sell_df_ab['age_bucket'] = ab_list
alternate_best_sell_df_ab['article_id'] = art_list
alternate_best_sell_df_ab['bestseller_rank'] = bs_list
alternate_best_sell_df_ab['RankType'] = rt_list

In [34]:
best_sell_df_ab = pd.concat([best_sell_df_ab.drop('prod_name', axis=1), 
                          alternate_best_sell_df_ab.drop('prod_name', axis=1)], axis=0)

best_sell_df_ab = best_sell_df_ab[~(best_sell_df_ab.article_id=='')] \
                .sort_values(['week','age_bucket','bestseller_rank'])

In [35]:
best_sell_df_ab.to_parquet('best_sell_with_agebucket.parquet')

In [36]:
## Find alternate item from same prod group with highest number of sale
## Find alternate item of lower price with a high number of sale
## Find alternate item of different color with highest number of sale
## Find alternate item of different grpahical appearance with highest number of sale

In [37]:
transactions.head()

t_dat      customer_id  article_id     price  sales_channel_id  week  \
0 2020-07-15  272412481300040   778064028  0.008458                 1    95   
1 2020-07-15  272412481300040   816592008  0.016932                 1    95   
2 2020-07-15  272412481300040   621381021  0.033881                 1    95   
3 2020-07-15  272412481300040   817477003  0.025407                 1    95   
4 2020-07-15  272412481300040   899088002  0.025407                 1    95   

   prod_name  colour_group_name  graphical_appearance_name age_bucket  
0        283                 11                          3      36-50  
1      20092                  2                          5      36-50  
2        935                  8                          0      36-50  
3       5423                 10                          4      36-50  
4      24059                  0                          0      36-50

In [38]:
## top 12 articles from last 10 weeks based on weekly sales

sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [39]:
best_sell_df_all = sales.reset_index()
best_sell_df_all = best_sell_df_all.merge(articles[['article_id','prod_name']], on='article_id', how='left')
best_sell_df_all['RankType'] = 'MostBought'

In [40]:
## From same channel find alternate item from same prod group with highest number of sale

def find_alternate_items_all(base_itm, base_wk, base_prod):

    item_1 = base_itm
    wk_num = base_wk
    prod_nm = base_prod
    
    item_2 = ''
    item_3 = ''
    item_4 = ''
    item_5 = ''

    color_1 = articles[articles.article_id==item_1]['colour_group_name'].iloc[0]
    g_1 = articles[articles.article_id==item_1]['graphical_appearance_name'].iloc[0]

    p1 = transactions[(transactions.article_id==item_1) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num)]['price'].mean()

    filter_df = transactions[(transactions.article_id!=item_1) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num)]

    if len(filter_df)>0:

        item_2 = filter_df.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
        color_2 = articles[articles.article_id==item_2]['colour_group_name'].iloc[0]
        g_2 = articles[articles.article_id==item_2]['graphical_appearance_name'].iloc[0]

        p2 = transactions[(transactions.article_id==item_2) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num)]['price'].mean()

        p_low = min(p1, p2)
        filter_df_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num) & 
                 (transactions.price < p_low)]

        if len(filter_df_2)>0:

            item_3 = filter_df_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
            color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
            g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            filter_df_3 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                     (transactions.prod_name==prod_nm) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.colour_group_name.isin([color_1, color_2, color_3]))]

            if len(filter_df_3)>0:
                item_4 = filter_df_3.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_4 = articles[articles.article_id==item_4]['colour_group_name'].iloc[0]
                g_4 = articles[articles.article_id==item_4]['graphical_appearance_name'].iloc[0]

                filter_df_4 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3, item_4])) & 
                     (transactions.prod_name==prod_nm) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3, g_4]))]

                if len(filter_df_4)>0:
                    item_5 = filter_df_4.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]            

            else:
                filter_df_3_1 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3]))]

                if len(filter_df_3_1)>0:
                    item_4 = filter_df_3_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

        else:
            filter_df_2_1 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.colour_group_name.isin([color_1, color_2]))]

            if len(filter_df_2_1)>0:
                item_3 = filter_df_2_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
                g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            else:
                filter_df_2_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                     (transactions.prod_name==prod_nm) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2]))]

                if len(filter_df_2_2)>0:
                    item_3 = filter_df_2_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

    return item_2, item_3, item_4, item_5


In [41]:
alternate_best_sell_df_all = pd.DataFrame(columns = best_sell_df_all.columns)

wk_list = []
art_list = []
bs_list = []
rt_list = []

for i in range(0, len(best_sell_df_all)):
    
    bs_itm = best_sell_df_all['article_id'][i]
    bs_wk = best_sell_df_all['week'][i]
    bs_prod = best_sell_df_all['prod_name'][i]
    bs_rank = best_sell_df_all['bestseller_rank'][i]
    
    a,b,c,d = find_alternate_items_all(bs_itm, bs_wk, bs_prod)

    wk_list.extend([bs_wk]*4)
    art_list.extend([a,b,c,d])
    bs_list.extend([bs_rank]*4)
    rt_list.extend(['SecondMost', 'LowerPrice', 'AnotherColor', 'AnotherAppearance'])

In [42]:
alternate_best_sell_df_all['week'] = wk_list
alternate_best_sell_df_all['article_id'] = art_list
alternate_best_sell_df_all['bestseller_rank'] = bs_list
alternate_best_sell_df_all['RankType'] = rt_list

In [43]:
best_sell_df_all = pd.concat([best_sell_df_all.drop('prod_name', axis=1), 
                          alternate_best_sell_df_all.drop('prod_name', axis=1)], axis=0)

best_sell_df_all = best_sell_df_all[~(best_sell_df_all.article_id=='')] \
                .sort_values(['week','bestseller_rank'])

In [44]:
best_sell_df_all.to_parquet('best_sell_all.parquet')